In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
os.environ['USE_PYGEOS'] = '0'
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics
from calitp_data.storage import get_fs

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

In [2]:
import segment_speed_utils

In [3]:
analysis_date = dt.date(2023, 4, 12)

In [4]:
fs = get_fs()

In [5]:
# path = 'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/export/avg_speeds_stop_segments_2023-04-12.parquet'

In [6]:
# fs.get(path, './avg_speeds_stop_segments_2023-04-12.parquet')

In [7]:
exported_segments = pd.read_parquet('./avg_speeds_stop_segments_2023-04-12.parquet')

In [8]:
exported_segments >> head(1)

,org_id,agency_name,shape_id,stop_sequence,stop_id,geometry,p50_mph,p20_mph,p80_mph,n_trips,time_of_day,base64_url,district,district_name
0,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,shp-10-09,2,55050,b'\x01\x02\x00\x00\x00\x1d\x00\x00\x00Q\x86\xb...,2.19,1.75,3.13,63,all_day,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3ZlaGljbG...,4,District 4 - Oakland


In [9]:
gdf = gpd.GeoDataFrame(exported_segments,
                 geometry=gpd.GeoSeries.from_wkb(exported_segments.geometry),
                 crs=segment_speed_utils.project_vars.PROJECT_CRS
                )

In [10]:
gdf >> head(3)

,org_id,agency_name,shape_id,stop_sequence,stop_id,geometry,p50_mph,p20_mph,p80_mph,n_trips,time_of_day,base64_url,district,district_name
0,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,shp-10-09,2,55050,"LINESTRING (-183825.823 -36469.556, -183825.82...",2.19,1.75,3.13,63,all_day,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3ZlaGljbG...,4,District 4 - Oakland
1,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,shp-10-09,2,55050,"LINESTRING (-183825.823 -36469.556, -183825.82...",2.20,1.71,2.41,24,peak,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3ZlaGljbG...,4,District 4 - Oakland
2,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,shp-10-09,3,51207,"LINESTRING (-183469.198 -36386.667, -183469.19...",10.08,7.09,16.05,61,all_day,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3ZlaGljbG...,4,District 4 - Oakland


In [11]:
# opening direct from gcs seemed to hang?
# gpd.read_file('./avg_speeds_stop_segments_2023-04-12.parquet')

In [12]:
shared_utils.rt_utils.get_speedmaps_ix_df?

Signature:
shared_utils.rt_utils.get_speedmaps_ix_df(
    analysis_date: datetime.date,
    itp_id: Optional[int] = None,
) -> pandas.core.frame.DataFrame
Docstring:
Collect relevant keys for finding all schedule and rt data for a reports-assessed organization.
Note that organizations may have multiple sets of feeds, or share feeds with other orgs.

Used with specific itp_id in rt_analysis.rt_parser.OperatorDayAnalysis, or without specifying itp_id
to get an overall table of which datasets were processed and how to deduplicate if needed
File:      ~/data-analyses/_shared_utils/shared_utils/rt_utils.py
Type:      function

In [13]:
shared_utils.rt_utils.get_speedmaps_ix_df(analysis_date) >> filter(_.organization_name.str.contains('Santa')) >> head(5)

,feed_key,schedule_gtfs_dataset_key,vehicle_positions_gtfs_dataset_key,organization_itp_id,caltrans_district,organization_name,service_date
23,199d652b518af1f2aef47e9c3c4f82e0,de792182088eecc3d5c0bd3f1df62965,33dd64366cfab802596d05a407a4a35a,293,05 - San Luis Obispo,Santa Barbara Metropolitan Transit District,2023-04-12
25,c82ceee91cd115a3c6a2b8bd6f939f68,73105f2d1cabc8170ab066d96863c5d5,3b7fb85888dc9a73f89b9a28272390b5,298,05 - San Luis Obispo,City of Santa Maria,2023-04-12
26,9621db94d9cef22bd14e817828f18540,8fa3380c9291d3694494c34b014642d0,4340d7ccf61d3a169068ffef1e310daf,301,04 - Oakland,City of Santa Rosa,2023-04-12
40,8b1e14f57472d6a2047dd501f28770c8,8f152d5328f38ce505eb9c647e08375e,64d857846a5263527036e2abd63e28df,295,07 - Los Angeles,City of Santa Clarita,2023-04-12
46,d979fbca8ab07423a0cd4d2d76ae1a45,dbbe8ee4864a2715a40749605395d584,6c2d7daaf979779fa2089c6395baf98b,300,07 - Los Angeles,City of Santa Monica,2023-04-12


In [14]:
samo_url = (tbls.mart_transit_database.dim_gtfs_datasets()
          >> filter(_.key == '6c2d7daaf979779fa2089c6395baf98b') # Santa Monica
          >> collect()
         ).base64_url.iloc[0]

In [15]:
segments_samo = (gdf >> filter(_.time_of_day == 'all_day', _.base64_url == samo_url)).copy()

In [16]:
segments_samo >> head(3)

,org_id,agency_name,shape_id,stop_sequence,stop_id,geometry,p50_mph,p20_mph,p80_mph,n_trips,time_of_day,base64_url,district,district_name
96104,recJHFDLpGRMIFgnL,City of Santa Monica,26156,2,361,"LINESTRING (141218.168 -446285.286, 141214.980...",8.31,2.89,14.55,5,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles
96106,recJHFDLpGRMIFgnL,City of Santa Monica,26156,3,362,"LINESTRING (140978.029 -445890.403, 140974.817...",14.25,14.25,14.25,1,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles
96107,recJHFDLpGRMIFgnL,City of Santa Monica,26156,4,363,"LINESTRING (140761.165 -445640.587, 140728.254...",5.20,5.20,5.20,1,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles


In [26]:
from shared_utils.rt_utils import try_parallel, arrowize_by_frequency, ZERO_THIRTY_COLORSCALE, SPEEDMAP_LEGEND_URL

In [19]:
segments_samo.geometry = segments_samo.geometry.apply(try_parallel)

In [20]:
segments_samo = segments_samo.apply(arrowize_by_frequency, axis=1, args=['n_trips'])

In [32]:
segments_samo >> head(3)

,org_id,agency_name,shape_id,stop_sequence,stop_id,geometry,p50_mph,p20_mph,p80_mph,n_trips,time_of_day,base64_url,district,district_name
96104,recJHFDLpGRMIFgnL,City of Santa Monica,26156,2,361,"POLYGON ((141143.668 -446163.831, 141063.982 -...",8.31,2.89,14.55,5,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles
96106,recJHFDLpGRMIFgnL,City of Santa Monica,26156,3,362,"POLYGON ((140784.705 -445643.610, 140783.928 -...",14.25,14.25,14.25,1,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles
96107,recJHFDLpGRMIFgnL,City of Santa Monica,26156,4,363,"POLYGON ((140795.451 -445611.377, 140773.422 -...",5.20,5.20,5.20,1,all_day,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdmVoaWNsZX...,7,District 7 - Los Angeles


## Segs to spa util

In [ ]:
## TODO ignore centroid warning

In [34]:
shared_utils.rt_utils.set_state_export(segments_samo)

/home/jovyan/data-analyses/_shared_utils/shared_utils/rt_utils.py:865: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/_shared_utils/shared_utils/rt_utils.py:865: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



writing to calitp-map-tiles/testing/test2.geojson.gz


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Map',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test2.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.02814634975405, -118.44655745935492),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdDIuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wMjgxNDYzNDk3NTQwNSwgLTExOC40NDY1NTc0NTkzNTQ5Ml0sICJ6b29tIjogMTN9'}

In [35]:
shared_utils.rt_utils.set_state_export(segments_samo, cmap=ZERO_THIRTY_COLORSCALE, legend_url=SPEEDMAP_LEGEND_URL,
                                      map_type='speedmap', color_col='p20_mph', filename='samo_segments_test')

/home/jovyan/data-analyses/_shared_utils/shared_utils/rt_utils.py:865: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/_shared_utils/shared_utils/rt_utils.py:865: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



writing to calitp-map-tiles/testing/samo_segments_test.geojson.gz


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Map',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/samo_segments_test.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.02814634975405, -118.44655745935492),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3Rpbmcvc2Ftb19zZWdtZW50c190ZXN0Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNSwgInRvb2x0aXBfc3BlZWRfa2V5IjogInAyMF9tcGgifSwgInR5cGUiOiAic3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM0LjAyODE0NjM0OTc1NDA1LCAtMTE4LjQ0NjU1NzQ1OTM1NDkyXSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiOiAiaHR0cHM6Ly9zdG9yYW

## Speedmap Comparision

In [ ]:
from rt_analysis import rt_filter_map_plot

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(300, analysis_date)

In [ ]:
_m = rt_day.segment_speed_map(no_render=True)

In [ ]:
rt_day.map_gz_export()

In [ ]:
rt_day.spa_map_state

In [ ]:
rt_day.spa_map_url

In [ ]:
## TODO shn?

In [ ]:
rt_day.display_spa_map()